## 📚 Prerequisites

Ensure that your Azure Services are properly set up, your Conda environment is created, and your environment variables are configured as per the instructions in the [SETTINGS.md](SETTINGS.md) file.

The PDFs used during this case study are publicly available:

- [Fisher EWD/EWS/EWT Valves through NPS 12x8 Instruction Manual](https://www.emerson.com/documents/automation/instruction-manual-fisher-ewd-ews-ewt-valves-through-nps-12x8-en-124788.pdf)
- [Fieldvue DVC6200 HW2 Digital Valve Controller Instruction Manual](https://www.emerson.com/documents/automation/instruction-manual-fieldvue-dvc6200-hw2-digital-valve-controller-en-123052.pdf)

These documents were chosen due to their complexity, particularly in terms of tables and interpretation of graphs. To follow along with this notebook, please download these files and upload them to your blob storage container.

## 📋 Table of Contents

This notebook guides you through the following sections:

1. [**Optical Character Recognition (OCR) with Azure AI Document Intelligence**](#optical-character-recognition-ocr-with-azure-ai-document-intelligence): Overview of Azure's Document Analysis Client and its pre-trained models for document analysis.

In [8]:
import os

# Define the target directory
target_directory = r"C:\Users\pablosal\Desktop\gbbai-azure-ai-document-intelligence"  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-azure-ai-document-intelligence


In [9]:
from src.ocr.document_intelligence import AzureDocumentIntelligenceManager

document_intelligence_client = AzureDocumentIntelligenceManager()

## Optical Character Recognition (OCR) with Azure AI Document Intelligence

Azure's Document Analysis Client provides a variety of pre-trained models that can be used to analyze documents. The `analyze_document` function takes in a document (either a URL or a file path) and the type of pre-trained model to use for analysis.

Here's a brief overview of the available pre-trained models:

- `'prebuilt-layout'`: This is the default model. It extracts text, tables, selection marks, and structure elements from the document.

- `'prebuilt-document'`: This model is used for generic document understanding.

- `'prebuilt-read'`: This model extracts both print and handwritten text.

- `'prebuilt-tax'`: This model is designed to process US tax documents.

- `'prebuilt-invoice'`: This model automates the processing of invoices.

- `'prebuilt-receipt'`: This model scans sales receipts for key data.

- `'prebuilt-id'`: This model processes identity documents.

- `'prebuilt-businesscard'`: This model extracts information from business cards.

- `'prebuilt-contract'`: This model analyzes contractual agreements.

- `'prebuilt-healthinsurancecard'`: This model processes health insurance cards.

In addition to these pre-trained models, Azure also offers custom and composed models. For more details, refer to the [Azure Document Intelligence Model Overview](https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/concept-model-overview?view=doc-intel-4.0.0).

The `analyze_document` function returns the analysis result, which can then be used for further processing or analysis.

In [10]:
# We will begin with the Fisher EWD/EWS/EWT Valves through NPS 12x8 Instruction Manual,
# which can be found at the following URL:
# https://www.emerson.com/documents/automation/instruction-manual-fisher-ewd-ews-ewt-valves-through-nps-12x8-en-124788.pdf
# We will use the 'prebuilt-layout' model for this task. This is the default model provided by Azure's Document Analysis Client,
# and it is capable of extracting text, tables, selection marks, and structure elements from the document.
# one  the latest feature is the abulity to extract content in a specific format, such as markdown.

document_url = "https://www.emerson.com/documents/automation/instruction-manual-fisher-ewd-ews-ewt-valves-through-nps-12x8-en-124788.pdf"
document_blob = "https://testeastusdev001.blob.core.windows.net/testretrieval/instruction-manual-fisher-et-eat-easy-e-valves-cl125-through-cl600-en-124782.pdf"
document_local = os.path.join(
    os.getcwd(),
    "utils\\data\\instruction-manual-fieldvue-dvc6200-hw2-digital-valve-controller-en-123052.pdf",
)
model_type = "prebuilt-layout"

result_ocr = document_intelligence_client.analyze_document(
    document_input=document_blob,
    model_type=model_type,
    output_format="markdown",
    features=["OCR_HIGH_RESOLUTION"],
    pages="1-3",
)

2024-03-06 23:04:48,224 - micro - MainProcess - INFO     Blob URL detected. Extracting content. (document_intelligence.py:analyze_document:148)


2024-03-06 23:04:49,303 - micro - MainProcess - INFO     Successfully downloaded blob file instruction-manual-fisher-et-eat-easy-e-valves-cl125-through-cl600-en-124782.pdf (blob_data_extractor.py:extract_content:88)


In [11]:
section_headings = [
    paragraph.content
    for paragraph in result_ocr.paragraphs
    if paragraph.role == "sectionHeading"
]

In [12]:
# retriving pages from the docuemnt
result_ocr.pages

[{'pageNumber': 1, 'angle': 0, 'width': 8.5, 'height': 11, 'unit': 'inch', 'words': [{'content': 'Instruction', 'polygon': [0.7911, 0.7322, 1.6174, 0.7248, 1.6148, 0.8767, 0.7852, 0.8735], 'confidence': 0.993, 'span': {'offset': 17, 'length': 11}}, {'content': 'Manual', 'polygon': [1.6695, 0.7242, 2.2367, 0.7164, 2.2367, 0.8767, 1.6671, 0.8767], 'confidence': 0.996, 'span': {'offset': 29, 'length': 6}}, {'content': 'D100398X012', 'polygon': [0.7845, 0.9163, 1.5, 0.9155, 1.5, 1.0282, 0.7845, 1.03], 'confidence': 0.993, 'span': {'offset': 36, 'length': 11}}, {'content': 'ET', 'polygon': [7.1741, 0.734, 7.3225, 0.7346, 7.3203, 0.8566, 7.172, 0.8567], 'confidence': 0.996, 'span': {'offset': 71, 'length': 2}}, {'content': 'Valve', 'polygon': [7.3485, 0.7347, 7.7033, 0.7344, 7.7033, 0.8567, 7.3464, 0.8564], 'confidence': 0.993, 'span': {'offset': 74, 'length': 5}}, {'content': 'August', 'polygon': [7.075, 0.919, 7.4297, 0.9209, 7.4272, 1.0383, 7.069, 1.04], 'confidence': 0.995, 'span': {'off

In [13]:
# retriving tables from the document
result_ocr.tables

[{'rowCount': 23, 'columnCount': 2, 'cells': [{'rowIndex': 0, 'columnIndex': 0, 'content': 'Introduction', 'boundingRegions': [{'pageNumber': 1, 'polygon': [0.704, 2.5798, 3.758, 2.5758, 3.758, 2.7056, 0.704, 2.7095]}], 'spans': [{'offset': 186, 'length': 12}], 'elements': ['/paragraphs/4']}, {'rowIndex': 0, 'columnIndex': 1, 'content': '1', 'boundingRegions': [{'pageNumber': 1, 'polygon': [3.758, 2.5758, 4.0253, 2.5758, 4.0253, 2.7056, 3.758, 2.7056]}], 'spans': [{'offset': 201, 'length': 1}], 'elements': ['/paragraphs/5']}, {'rowIndex': 1, 'columnIndex': 0, 'content': 'Scope of Manual', 'boundingRegions': [{'pageNumber': 1, 'polygon': [0.704, 2.7095, 3.758, 2.7056, 3.762, 2.8865, 0.704, 2.8983]}], 'spans': [{'offset': 207, 'length': 15}], 'elements': ['/paragraphs/6']}, {'rowIndex': 1, 'columnIndex': 1, 'content': '1', 'boundingRegions': [{'pageNumber': 1, 'polygon': [3.758, 2.7056, 4.0253, 2.7056, 4.0292, 2.8826, 3.762, 2.8865]}], 'spans': [{'offset': 225, 'length': 1}], 'elements':

In [14]:
print(result_ocr.content)

<!-- PageHeader="Instruction Manual D100398X012" -->

<!-- PageHeader="ET Valve August 2021" -->

Fisher™ ET and EAT easy-e™ Valves CL125 through CL600
===


# Contents

|||
| - | - |
| Introduction | 1 |
| Scope of Manual | 1 |
| Description | 3 |
| Specifications | 3 |
| Educational Services | 3 |
| Installation | 4 |
| Maintenance | 5 |
| Packing Lubrication | 6 |
| Packing Maintenance | 6 |
| Replacing Packing | 7 |
| Trim Maintenance | 13 |
| Disassembly | 13 |
| Lapping Metal Seats | 15 |
| Valve Plug Maintenance | 16 |
| Assembly | 18 |
| ENVIRO-SEAL Bellows Seal Bonnet | 20 |
| Replacing a Plain or Extension Bonnet with an ENVIRO-SEAL Bellows Seal Bonnet (Stem/Bellows Assembly) | 20 |
| Replacement of an Installed ENVIRO-SEAL Bellows Seal Bonnet | |
| (Stem/Bellows Assembly) | 23 |
| Purging the ENVIRO-SEAL Bellows Seal Bonnet . | 25 |
| Parts Ordering | 25 |
| Parts Kits | 26 |
| Parts List | 31 |

<figure>

![](figures/0)

<!-- FigureContent="Figure 1. Fisher ET Control Valve